## Certificate Project - 1 : [Intel image classification](https://www.kaggle.com/puneet6060/intel-image-classification)

# Import libraries:

In [ ]:
import os
import glob
import numpy as np

from PIL import Image,ImageOps
import matplotlib.pyplot as plt
%matplotlib inline

# List out directories

In [ ]:
base_dir = '../input/intel-image-classification/'
directory = os.listdir(base_dir)
directory

# Prepare training & testing directory paths

In [ ]:
train_dir = base_dir + 'seg_train/seg_train/'
test_dir = base_dir + 'seg_test/seg_test/'

# Prepare CLASSES array

In [ ]:
# Hint: 
#   You are free to use api available in 'os' library, which returns array of directories inside given path.
#   Or, you can manually prepare the array, as CLASSES = ['folder name1', 'folder name 2' ...]
CLASSES = [x for x in os.listdir('../input/intel-image-classification/seg_train/seg_train')]
print(CLASSES)

# Total no. of images per class in training dataset

In [ ]:
# TRAINING DIR: For each class, lets see how many images are there..
for imgType in CLASSES:
    imgTypePath = train_dir + "/" + imgType + "/"
    print("CLASS: " + imgType + ", Total images: " + str(len(os.listdir(imgTypePath)))) 

# Total no. of images per class in testing dataset

In [ ]:
# TESTING DIR: For each class, lets see how many images are there..
for imgType in CLASSES:
    imgTypePath = test_dir + "/" + imgType + "/"
    print("CLASS: " + imgType + ", Total images: " + str(len(os.listdir(imgTypePath))))

# Pipeline helper functions

In [ ]:
def convert_to_grayscale(img):
    #Convert to grayscale
    return ImageOps.grayscale(img) #Return gray scale image object using ImageOps.

def reshape_img(img, target_size=(150,150)):
    #Reshape any image to a fixed shape
    return img.resize(target_size, Image.ANTIALIAS) #Hint: Use resize() API on image object and pass target size param.

def display_numpy_img(np_img, img_name="Transformed image"):
    plt.figure(figsize = (6,6))
    plt.imshow(np_img, cmap='gray')
    plt.title(img_name)
    
def transform_image(img_file_path):
    img_obj = Image.open(img_file_path)
    #print(img_obj.format)
    #print(np.array(img_obj).shape)
    #Perform transformations in series
    img_obj = convert_to_grayscale(img_obj)
    img_obj = reshape_img(img_obj, (150,150))
    np_arr_img = np.array(img_obj)
    return np_arr_img

def load_dir_to_numpy(dir_path, maxImgs=1500):
    file_list = glob.glob(dir_path+'/*')
    imgs = []
    #Load image by image
    imgCount=0
    for fname in file_list:
        if imgCount>=maxImgs:
            break
        img_np = transform_image(fname)
        imgs.append(img_np)
        imgCount = imgCount + 1
    np_imgs = np.array(imgs)
    return np_imgs

def prepare_image_data(dir_path, MAX_IMGS):
    imgs_arr_X = []
    data_arr_y = []
    classIdx = 0;
    for imgType in CLASSES:
        IMG_DIR = dir_path + "/" + imgType + "/"
        #  print("IMG_DIR: " + IMG_DIR)
        imgs_arr = load_dir_to_numpy(IMG_DIR, MAX_IMGS)
        #print(imgType + ": " + str(imgs_arr.shape))
        imgs_arr_X.extend(imgs_arr)
        data_y = np.full((imgs_arr.shape[0],1), classIdx)
        data_arr_y.extend(data_y)
        classIdx += 1
    np_img_arr_X = np.array(imgs_arr_X)
    np_data_arr_y = np.array(data_arr_y)
    return np_img_arr_X,np_data_arr_y

# Prepare training dataset

In [ ]:
train_np_x,train_np_y = prepare_image_data(train_dir, 500)
print('train_np_x.shape:', train_np_x.shape)
print('train_np_y.shape:', train_np_y.shape)

# Flatten out the 2D image data into 1D vector

In [ ]:

train_size = train_np_x.shape[0]
train_np_x = train_np_x.reshape((train_size, -1))
print('After reshaping, train_np_x.shape:', train_np_x.shape)

# Import & Prepare the model object

In [ ]:
# Import a model
# Train and eval
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
# Using default hyperparameters
#model = DecisionTreeClassifier(criterion="entropy") #Hint: Any algorithm say sklearn.linear_model.SGDClassifier OR sklearn.tree.DecisionTreeClassifier() etc..
#model=GaussianNB()
model=RandomForestRegressor(n_estimators = 1, random_state = 10)

# Train the model

In [ ]:
model.fit(train_np_x, train_np_y.reshape(-1))

# Prepare testing data

In [ ]:
test_np_x,test_np_y = prepare_image_data(test_dir, 200)

test_size = test_np_x.shape[0]
test_np_x = test_np_x.reshape((test_size, -1))
print('Test shape:', test_np_x.shape)

# Predict using testing data

In [ ]:
# Get predicted values for 'test_np_x' using trained 'model' 
predicted_y = model.predict(test_np_x)

# What's the trained model accuracy on test data?

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
actual_y = test_np_y.reshape(-1)
print('Accuracy Score:', accuracy_score(actual_y,predicted_y))

# Model training performance report

In [ ]:
print(classification_report(actual_y,predicted_y))

# Confusion matrix

In [ ]:
print(confusion_matrix(actual_y,predicted_y))